# Using Row Access Policies based on the Current User in Streamlit in Snowflake
This Notebook will help set up a Row Access Policy (RAP) to restrict data access based on the current user when using Streamlit in Snowflake.

The key to this is the `READ SESSION` permission. If the owner role of the Streamlit has _not_ been granted the `READ SESSION` permission, then the session context functions (e.g., `CURRENT_USER()`, `CURRENT_AVAILABLE_ROLES()`, etc) return `NULL`. But if the owner role of the Streamlit _has_ been granted this permission, then those context functions return the proper values. 

:orange[Note:] `CURRENT_ROLE()` will always return the owner role of the Streamlit.

In this Notebook we:
1. create a role to own the Streamlit and a role that will be allowed to use the Streamlit, and create 3 users and grant each the role that will be allowed to use the Streamlit
2. create a data table with a column we will use to restrict row access (i.e., the `REGION_ID` column)
3. Create an entitlements table that maps which users can see which region(s)
4. create a UDF and RAP based on the current user, and apply that to the data table
5. test data access
6. create a Streamlit in Snowflake and grant usage to the Streamlit user role


## 1. Create the necessary roles and users
First we create the role that will own the Streamlit. We also grant the role to the current user. This role also needs the `READ SESSION` permission.

We will also create 3 users (`USER1`, `USER2`, and `USER3`). We create another role that will get usage permission on the Streamlit, and grant that role `RL_APP_USER` to the current user and the 3 new users.

In [ ]:
USE ROLE securityadmin;
SET cur_user = CURRENT_USER();

CREATE ROLE IF NOT EXISTS rl_app;
GRANT ROLE rl_app TO USER IDENTIFIER($cur_user);
GRANT READ SESSION ON ACCOUNT TO ROLE rl_app;

CREATE ROLE IF NOT EXISTS rl_app_user;
GRANT ROLE rl_app_user TO USER IDENTIFIER($cur_user);

CREATE USER IF NOT EXISTS user1 PASSWORD='abc123' DEFAULT_ROLE = rl_app_user DEFAULT_SECONDARY_ROLES = ('ALL') MUST_CHANGE_PASSWORD = TRUE;
CREATE USER IF NOT EXISTS user2 PASSWORD='abc123' DEFAULT_ROLE = rl_app_user DEFAULT_SECONDARY_ROLES = ('ALL') MUST_CHANGE_PASSWORD = TRUE;
CREATE USER IF NOT EXISTS user2 PASSWORD='abc123' DEFAULT_ROLE = rl_app_user DEFAULT_SECONDARY_ROLES = ('ALL') MUST_CHANGE_PASSWORD = TRUE;
GRANT ROLE rl_app_user TO USER user1;
GRANT ROLE rl_app_user TO USER user2;
GRANT ROLE rl_app_user TO USER user3;

## 2. Create the Data Table
Next we create the data table that we will later add a Row Access Policy to restrict data access based on the current user. This data is accessible by the role that owns the Streamlit.

In [ ]:
USE ROLE accountadmin;
CREATE DATABASE IF NOT EXISTS sandbox;
CREATE SCHEMA IF NOT EXISTS sandbox.idea;
GRANT USAGE ON DATABASE sandbox TO ROLE rl_app;
GRANT ALL ON SCHEMA sandbox.idea TO ROLE rl_app;
GRANT USAGE ON DATABASE sandbox TO ROLE rl_app_user;
GRANT USAGE ON SCHEMA sandbox.idea TO ROLE rl_app_user;

USE ROLE rl_app;
USE SCHEMA sandbox.idea;
DROP TABLE IF EXISTS app_data;
CREATE TABLE app_data(a INT, b DOUBLE, c TEXT, region_id INT);
INSERT INTO app_data(a, b, c, region_id) VALUES 
    (101, 101.101, 'one hundred one', 1),
    (102, 102.102, 'one hundred two', 1),
    (103, 103.103, 'one hundred three', 1),
    (201, 101.101, 'two hundred one', 2),
    (202, 202.202, 'two hundred two', 2),
    (203, 203.203, 'two hundred three', 2),
    (301, 301.301, 'three hundred one', 3),
    (302, 302.302, 'three hundred two', 3),
    (303, 303.303, 'three hundred three', 3),
    (901, 901.901, 'nine hundred one', 9),
    (902, 902.902, 'nine hundred two', 9),
    (903, 903.903, 'nine hundred three', 9)
;
GRANT SELECT ON TABLE app_data TO ROLE rl_app_user;

## 3. Create the Entitlements Table
Next we will create the entitlements table that will map which users have access to which region IDs. Additionally, we will allow the `CORTEX_USER_ROLE` to have access to all of the data, so we add a row in the entitlement table for each region ID.

In [ ]:
USE ROLE rl_app;
USE SCHEMA sandbox.idea;

DROP TABLE IF EXISTS ent_region;
CREATE TABLE ent_region(key STRING, value INT);

-- Let's you to access all of the data
INSERT INTO ent_region
    SELECT CURRENT_USER() AS key, region_id AS value 
    FROM app_data 
    GROUP BY region_id
;

-- Now, let's restrict tenants to just their region
INSERT INTO ent_region(key,value) VALUES 
    ('USER1', 1), 
    ('USER2', 2), 
    ('USER3', 3)
;

SELECT * FROM ent_region ORDER BY key, value;

## 4. Set up the Row Access Policy
In order to make the Row Access Policy performant, we will create a memoizable function that collects all `VALUE`s that a given `KEY` has access to. Then, we create a Row Access Policy that uses that function, passing in the `CURRENT_USER()` as the argument. Since this is contant for the length of the query, the UDF will run once and the result reused (as is the behavior of memoizable functions).

In [ ]:
USE ROLE rl_app;
USE SCHEMA sandbox.idea;

DROP FUNCTION IF EXISTS udf_regions(STRING);
CREATE FUNCTION udf_regions(ikey STRING)
    RETURNS ARRAY
    MEMOIZABLE
    AS 'SELECT ARRAY_AGG(value) FROM ent_region WHERE key = ikey';

DROP ROW ACCESS POLICY IF EXISTS rap_region_user;
CREATE ROW ACCESS POLICY rap_region_user
    AS (region_id INT)
    RETURNS BOOLEAN ->
        ARRAY_CONTAINS(region_id, udf_regions(CURRENT_USER()))
;

Now we add this Row Access Policy to the `REGION_ID` column of the `APP_DATA` table.

In [ ]:
USE ROLE rl_app;
USE SCHEMA sandbox.idea;

ALTER TABLE app_data DROP ALL ROW ACCESS POLICIES; -- Just to make sure there aren't any others
ALTER TABLE app_data ADD ROW ACCESS POLICY rap_region_user ON (region_id);

## 5. Testing
Now we can test the access to the `APP_DATA` table. First, let's use the current user and ensure that all data is accessible.

In [ ]:
USE ROLE rl_app;
USE SCHEMA sandbox.idea;
SELECT region_id, COUNT(*) FROM app_data GROUP BY region_id;

Next, let's use user `USER1` to see that only data from region `1` is accessible. To do this, you will need to log out of Snowsight and log in as `USER1`. In a worksheet run the following:

In [ ]:
USE ROLE rl_app_user;
USE SCHEMA sandbox.idea;
SELECT region_id, COUNT(*) FROM app_data GROUP BY region_id;

Feel free to try this with user `USER2` (to see data from region `2`), and `USER3` (to see data from region `3`).

## 6. Application
At this point, we can turn our attention to a Streamlit in Snowflake that will only allow users to access the data in their allowed region(s). We will do this using the Snowsight UI.

1. Open a new tab/window (or clone this tab), log into Snowflake, and navigate to the "Projects" sidebar and the "Streamlit" page.
2. Make sure you have selected the `RL_APP` role as your current role. You can do this by clicking your initials in the lower left, choose the "Switch Role" option, and select the `RL_APP` role.
3. Click the "+ Streamlit App" button to create a new app. You can name it whatever you would like (might I suggest "RAP Current User"). Choose the `SANDBOX` database and `IDEA` schema, and choose an app warehouse.
4. In the editor pane, delete all of the code and replace it with the following:
```python
# Import python packages
import streamlit as st
from snowflake.snowpark.context import get_active_session

# Get the current credentials
session = get_active_session()

# Write directly to the app
st.title(f"Using RAP with SIS")

user = session.sql("SELECT current_user() AS user").collect()[0]['USER']
st.markdown(f"Hello. Your current user is: :blue[{user}]")

st.markdown("Let's access the `APP_DATA` table")
df = session.table("sandbox.idea.app_data").to_pandas()
st.dataframe(df)
```

We now have a Streamlit that will show the current user and select data from the `APP_DATA` table. Running it as yourself you will see that you can see all the data.

Now we need to share the Streamlit app with the `RL_APP_USER` role so that our other users can access it.

5. Click the "Share" button in the upper right. In the dialog box, click the selectbox named "Add role to share with", enter `RL_APP_USER` and select it. Click "Done".

At this point, you can visit the Streamlit as `USER1`, `USER2`, or `USER3` and see that only some of the data is visible, specifically the data they have been granted entitlement to in our `ENT_REGION` table. Do this by logging out and back in as one of those users, visit the "Projects" sidebar and "Streamlit" page. Ensure that you have selected the `RL_APP_USER` as your current role (if not, follow the same instructions as step 2 above to change roles). Click on the Streamlit that you created and shared and you should see that the Stremalit identifies your current user and only shows you the data for the region that you have been entitled to see.